<a href="https://colab.research.google.com/github/Sopralapanca/medium-articles/blob/main/Telegram/%5BMedium%5DTelethon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all go to [my.telegram.org](https://my.telegram.org), and login with your telephone number.
Create new application, enter app title and short name, url can be left empty. Choose a platform and click on create application button to obtain *app api_id* and *app api_hash*

In [1]:
!pip install Telethon==1.36.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.3/606.3 kB 13.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for Telethon: filename=Telethon-1.36.0-py3-none-any.whl size=680879 sha256=ba273a3581380a1156a4c3407d491dc0a6a2f551fb940b3a389d2242d3437faf
  Stored in directory: /root/.cache/pip/wheels/4f/6f/86/da2c68242ca7dd92786186731ee763b7ecac8866666ac47170
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26346 sha256=6ee03b6448a18cbd4bf9130c2e87733d2ba9c3601b3f078d3d473c3fd53ac899
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built Telethon pyaes


In [12]:
# to run the script in google colab, this can be avoided if you are running the code in your pc
!pip install nest_asyncio

import nest_asyncio

# Apply the patch to allow nested event loops
nest_asyncio.apply()

In [8]:
api_id = "YOUR API ID"
api_hash = "YOUR API HASH"
phone = "YOUR PHONE NUMBER WITH +xxx"

In [ ]:
from telethon import TelegramClient, events
import asyncio
import os
from sqlite3 import OperationalError

channels_list = ["channel1", "channel2"]
my_channel = "my_channel"

async def main(tg_client):
  print("starting...")
  await tg_client.start(phone)

  @tg_client.on(events.NewMessage(incoming=True, from_users=channels_list))
  async def my_event_handler(event):

      sender = await event.get_sender()

      if sender.username in channels_list:
          channel = sender.username

          # get last message of the channel
          async for message in tg_client.iter_messages(channel, limit=1):
              print(message.text)

              img_path = "./imgs/image.jpg"
              await tg_client.download_media(message=message.media, file=img_path)

              await tg_client.send_file(entity=my_channel, file=img_path, caption="My Caption")


  await tg_client.run_until_disconnected()


if __name__ == '__main__':

    telethon_session_name = 'telethon_session'
    session_file = f'./{telethon_session_name}.session'

    try:
      telegram_client = TelegramClient(session_file, api_id, api_hash)

    except OperationalError:
      # if you have login problem like locked database, try to delete it
      if os.path.exists(session_file):
          os.remove(session_file)
          telegram_client = TelegramClient(session_file, api_id, api_hash)
      else:
          raise

    try:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(
            main(tg_client=telegram_client))
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # If you run the script in google colab leave the following code commented, otherwhise remove comments and delete pass
        #loop.run_until_complete(asyncio.sleep(2))
        #loop.close()
        pass

Check the list of all the events https://docs.telethon.dev/en/stable/modules/events.html